In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

dataset_path = 'dataset/Train'

label_types = os.listdir('dataset/Train')
print (label_types)  

['Correct', 'Incorrect', 'Unmoving']


In [2]:
# rooms = []

# for item in label_types:
#     # Get all the file names
#     all_rooms = os.listdir(dataset_path + '/' +item)

#     # Add them to the list
#     for room in all_rooms:
#         if os.path.isfile(dataset_path + '/' + item + '/' + room):
#             rooms.append((item, f'{dataset_path}/{item}/{room}'))
#         else:
#             print(dataset_path + '/' + item + '/' + room)
    
# # Build a dataframe        
# train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
# print(train_df.head())
# print(train_df.tail())

In [3]:
# df = train_df.loc[:,['video_name','tag']]
# df
# df.to_csv('train.csv')

In [4]:
# dataset_path = 'dataset/Test'

# room_types = os.listdir('dataset/Test')
# print("Types of activities found: ", len(room_types))

# rooms = []

# for item in room_types:
#     # Get all the file names
#     all_rooms = os.listdir(dataset_path + '/' + item)

#     # Add them to the list
#     for room in all_rooms:
#         if os.path.isfile(dataset_path + '/' + item + '/' + room):
#             rooms.append((item, f'{dataset_path}/{item}/{room}'))
#         else:
#             print(dataset_path + '/' + item + '/' + room)
    
# # Build a dataframe        
# test_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
# print(test_df.head())
# print(test_df.tail())

# df = test_df.loc[:,['video_name','tag']]
# df
# df.to_csv('test.csv')

In [5]:
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [6]:
# show tf version
print(tf.__version__)

2.10.0


In [7]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
# if gpus:
#   try:
#     tf.config.experimental.set_virtual_device_configuration(
#         gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
#   except RuntimeError as e:
#     print(e)
# print(gpus)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [8]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")


Total videos for training: 124
Total videos for testing: 124


In [9]:
# The following two methods are taken from this tutorial:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
IMG_SIZE = 224


def crop_center_square(frame, boundingBox):
    x, y, w, h = [round(float(x)) for x in boundingBox]
    min_dim = min(w, h)
    start_x = x + w // 2 - min_dim // 2
    start_y = y + h // 2 - min_dim // 2
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, boundingBox, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame, boundingBox)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [10]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

In [11]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["trueLabel"]))
print(label_processor.get_vocabulary())

labels = train_df["trueLabel"].values
labels = label_processor(labels[..., None]).numpy()
labels

['Correct', 'Incorrect', 'Unmoving']


array([[0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [2],
       [2],
       [2],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [2],
       [2],
       [2],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [2],
       [2],
       [2],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [2],
       [2],
       [2],
       [1],
       [1],
       [1],
       [1],
       [1],
       [2],
       [2],
       [2],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [2],
    

In [12]:
#Define hyperparameters

IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [13]:
def prepare_all_videos(df):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    
    ##take all classlabels from train_df column named 'tag' and store in labels
    labels = df["trueLabel"].values
    
    #convert classlabels to label encoding
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool") # 145,20
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32") #145,20,2048

    # Iterate over datatframe rows (i.e. videos) and store features for each video
    for idx, row in df.iterrows():
        # Get full video path from relative video path.
        path = os.path.abspath(row.video_name)
        # Get bounding boxes for frames in video
        bounding_X = row.boundingBoxX
        bounding_Y = row.boundingBoxY
        bounding_W = row.boundingBoxWidth
        bounding_H = row.boundingBoxHeight
        # Gather all its frames and add a batch dimension.
        frames = load_video(path, boundingBox=(bounding_X, bounding_Y, bounding_W, bounding_H))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df)
test_data, test_labels = prepare_all_videos(test_df)

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")



print(f"train_labels in train set: {train_labels.shape}")

print(f"test_labels in train set: {test_labels.shape}")

# MAX_SEQ_LENGTH = 20, NUM_FEATURES = 2048. We have defined this above under hyper parameters

1/1 [==============================] - 0s 28ms/step
Frame features in train set: (124, 20, 2048)
Frame masks in train set: (124, 20)
train_labels in train set: (124, 1)
test_labels in train set: (124, 1)


In [18]:
# Utility for our sequence model.
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

EPOCHS = 30
# Utility for running experiments.
def run_experiment():
    filepath = "./tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        # callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/30


InvalidArgumentError: Graph execution error:

No OpKernel was registered to support Op 'CudnnRNNV3' used by {{node cond_41/then/_0/cond/CudnnRNNV3}} with these attrs: [seed=0, dropout=0, T=DT_FLOAT, input_mode="linear_input", direction="unidirectional", rnn_mode="gru", is_training=true, seed2=0, num_proj=0, time_major=false]
Registered devices: [CPU, GPU]
Registered kernels:
  <no registered kernels>

	 [[cond_41/then/_0/cond/CudnnRNNV3]]
	 [[model_3/gru_6/PartitionedCall]] [Op:__inference_train_function_385538]

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True